#Dataset preparation

##Import dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown 1Qb4UrNsjvlU-wEsR9d7rckB0YS_LXgb2
!unzip Cityscapes.zip -d cityscapes/

Output streaming troncato alle ultime 5000 righe.
  inflating: cityscapes/Cityscapes/Cityspaces/gtFine/train/strasbourg/strasbourg_000000_029577_gtFine_labelTrainIds.png  
  inflating: cityscapes/Cityscapes/Cityspaces/gtFine/train/strasbourg/strasbourg_000000_029729_gtFine_color.png  
  inflating: cityscapes/Cityscapes/Cityspaces/gtFine/train/strasbourg/strasbourg_000000_029729_gtFine_labelTrainIds.png  
  inflating: cityscapes/Cityscapes/Cityspaces/gtFine/train/strasbourg/strasbourg_000000_029839_gtFine_color.png  
  inflating: cityscapes/Cityscapes/Cityspaces/gtFine/train/strasbourg/strasbourg_000000_029839_gtFine_labelTrainIds.png  
  inflating: cityscapes/Cityscapes/Cityspaces/gtFine/train/strasbourg/strasbourg_000000_029915_gtFine_color.png  
  inflating: cityscapes/Cityscapes/Cityspaces/gtFine/train/strasbourg/strasbourg_000000_029915_gtFine_labelTrainIds.png  
  inflating: cityscapes/Cityscapes/Cityspaces/gtFine/train/strasbourg/strasbourg_000000_030017_gtFine_color.png  
  infl

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/MLDL_repo/step2')

In [3]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=7fdce295a95ee46b6b9a4a6781e878a5c513bbb4591528249c02e3943cc73ad3
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=f62a52d6add647dbcb5c681f7a81eb7277fbe68abac5f55f2951596ac0b375c4
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from datasets_custom.cityscapes import CityScapes
from models.bisenet.build_bisenet import BiSeNet
from utils_p import *
from train import *

def main():
    # dataset definitions and transformation
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

    target_transform = transforms.Compose([
        transforms.Resize((512, 1024), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

    dataset_root = '/content/cityscapes/Cityscapes/Cityspaces'

    train_dataset = CityScapes(root=dataset_root, split='train', transform=transform, target_transform=target_transform)
    print(f"Train dataset size: {len(train_dataset)}")

    test_dataset = CityScapes(root=dataset_root, split='val', transform=transform, target_transform=target_transform)
    print(f"Test dataset size: {len(test_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=2)

    # model and hyperparameters
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    num_classes = 19
    base_lr = 2.5e-4
    batch_size = 2
    epochs = 50
    crop_size = (512, 1024)
    context_path = 'resnet18'

    model = BiSeNet(num_classes = num_classes, context_path= context_path)

    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)


    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_miou = 0.0

    # Training
    for epoch in range(epochs):
        train_one_epoch(model, train_loader, optimizer, base_lr, epoch, epochs, device)


    # save model
    torch.save(model.state_dict(), '/content/drive/MyDrive/MLDL_repo/step2/bisenet_resnet18_1.pth')
    print("Model saved")

    # Validation
    validate(model, test_loader, num_classes = 19, device = device , best_miou = 0.0)


if __name__ == "__main__":
    main()

Loaded 1572 images for split: train
Train dataset size: 1572
Loaded 500 images for split: val
Test dataset size: 500


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 228MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 206MB/s]



Epoch 1/50


100%|██████████| 786/786 [01:35<00:00,  8.24it/s]


 Avg Training Loss: 0.7029

Epoch 2/50


...

 Avg Training Loss: 0.0460

Epoch 50/50


100%|██████████| 786/786 [01:32<00:00,  8.49it/s]


 Avg Training Loss: 0.0456
Model saved


🔍 Validating: 100%|██████████| 250/250 [00:30<00:00,  8.31it/s]

 Validation mIoU: 0.5581
 New best mIoU found!


In [4]:
# model stats
from models.bisenet.build_bisenet import *
from utils_p import *

# initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# initialize model
num_classes = 19
context_path = 'resnet18'

model = BiSeNet(num_classes=num_classes, context_path=context_path)

#load model
model_path = r'/content/drive/MyDrive/MLDL_repo/step2/bisenet_resnet18_1.pth'
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model = model.to(device).eval() # python built in function

results = {}

analyze_flops(model, height=512, width=1024)
results = benchmark_model(model, height = 512, width = 1024, iterations=1000)
print(results)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 234MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 231MB/s]


| module                                      | #parameters or shape   | #flops     |
|:--------------------------------------------|:-----------------------|:-----------|
| model                                       | 12.582M                | 25.78G     |
|  saptial_path                               |  0.371M                |  5.088G    |
|   saptial_path.convblock1                   |   1.856K               |   0.243G   |
|    saptial_path.convblock1.conv1            |    1.728K              |    0.226G  |
|    saptial_path.convblock1.bn               |    0.128K              |    16.777M |
|   saptial_path.convblock2                   |   73.984K              |   2.424G   |
|    saptial_path.convblock2.conv1            |    73.728K             |    2.416G  |
|    saptial_path.convblock2.bn               |    0.256K              |    8.389M  |
|   saptial_path.convblock3                   |   0.295M               |   2.42G    |
|    saptial_path.convblock3.conv1            |    0.2